In [ ]:
import Maxvol
import Random
import StatsBase
import LinearAlgebra as LA

In [ ]:
function _maxvol(A)
    m, n = size(A)
    @assert size(A, 1) >= size(A, 2)
    A_ = copy(A)
    piv, niters = Maxvol.maxvol!(A_)
    return sort(piv)
end

function pseudoskelton(A::Matrix{T}, k::Int; niter=10) where T
    m, n = size(A)

    # Selected columns
    J = sort(StatsBase.sample(1:n, k, replace=false))

    C = Matrix{T}(undef, m, k)
    M = Matrix{T}(undef, k, k)
    R = Matrix{T}(undef, k, n)

    det_prev = nothing
    for iter in 1:niter
        C = A[:, J]

        # Select rows by maxvol
        I = _maxvol(C);

        M .= A[I, J]
        R .= A[I, :]

        det_M = abs(LA.det(M))
        if !isnothing(det_prev) && abs(det_M - det_prev) < 1e-5 * abs(det_prev)
            break
        end
        det_prev = det_M

        # Reselect columns by maxvol
        J = _maxvol(transpose(R))
    end

    F = LA.svd(M)

    L = C * F.V * LA.Diagonal(1 ./ F.S) * F.U'

    return L, R
end

In [ ]:
m, n = 20, 20
k = 2

x = LinRange(0, 1, m)
y = LinRange(0, 1, n)

A = Array{Float64,2}(undef, m, n)
for j in 1:n, i in 1:m
    A[i, j] = x[i]^2 + y[j]^2 
end

L, R = pseudoskelton(A, k)
;

In [ ]:
LA.norm(A - L * R)/LA.norm(A)